In [ ]:
%run ../widgets/config_check.py

In [ ]:
# <api>
import numpy as np
from scipy.stats import norm
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
%matplotlib inline

# Load classes and functions from the previous parts
from jupyter_cms.loader import load_notebook
smpl_intro = load_notebook('./Sampling_Intro.ipynb')
smpl_index = load_notebook('./Sampling_Index.ipynb')

In [ ]:
smpl_index.help()

# Rejection sampling

The next sampling method is less efficient, but more general in the sense that it can be used even if the normalization constant of the desired density $p(x)$ is unavailable.

It is based on the idea that a sample at position $x$ should be drawn with probability $p(x)$, i.e. the height under the graph of $x$. Thus, a sample can be drawn using two random numbers as follows:

* Draw a sample $x$ from some distribution $q(x)$
* Draw a uniform number $u$ between 0 and $c q(x)$:
  - If $u < p(x)$ return the sample $x$
  - Else discard $x$ and repeat
  
Here, it is assumed that samples from $q(x)$ can be drawn efficiently. Further, the scaling constant $c$ must be choosen such that $c q(x) \geq p(x) \quad \forall x$. The plot below illustrates the idea for the Gaussian distribution:

In [ ]:
# <api>
def p(x):
    """
    Unnormalized density of the standard Gaussian
    """
    return np.exp(-0.5*x**2)

In [ ]:
q = norm(loc=0, scale=2)
x = np.arange(-5,5,0.01)
plt.plot(x, p(x), 'b-', x, 5.1*q.pdf(x), 'g-')
plt.xlabel('x', fontsize=15)
plt.ylabel('Unnormalized probability', fontsize=15)
plt.legend(['Density p(x)', 'Envelope c*q(x)'], fontsize=10, loc=1)
plt.title('Illustration of proposal distribution', fontsize=15);

In [ ]:
#api
class RejectionSampling (smpl_intro.Sampling):
    def __init__ (self, p, q, c):
        """
        q is assumed to support sampling q.rvs and density evaluation q.pdf
        """
        self.p = p
        self.q = q
        self.c = c
        self.tries = 0
        self.samples = 0
        self.last_approved = [0,0]

    def sample (self):
        while True:
            self.tries += 1
            x = self.q.rvs()
            u = np.random.uniform(low=0, high=self.c*self.q.pdf(x))
            if u < self.p(x):
                self.samples += 1
                return x
            
    def sample_all (self):
        while True:
            self.tries += 1
            x = self.q.rvs()
            u = np.random.uniform(low=0, high=self.c*self.q.pdf(x))
            if u < self.p(x):
                self.samples += 1
                self.last_approved = x
                return [x,x]
            else:
                return [self.last_approved, x]
            
    def __str__ (self):
        return "Rejection sampling: %d tries for %d samples" % (self.tries, self.samples)           

In [ ]:
sampling = RejectionSampling(p=p, q=norm(loc=0, scale=2), c=5.1)

smpl_intro.show_sampling(sampling, plotter=smpl_intro.gauss_hist, 
                    f_exp=lambda x: np.power(x,2), true_exp=1.0)
print (sampling)

Rejection sampling tends to work well, if a distribution $q(x)$ which closely approximates $p(x)$ from above can be found. Otherwise, the area between the curves will be large leading to a high rejection rate. Especially in high dimensions this is a severe problem.

A standard example to illustrate the difficulties arising in high dimensions is a 2-dimensional Gaussian with high correlation:

In [ ]:
# <api>
p2d = multivariate_normal(mean=[0,0], cov=[[1, 0.99],[0.99, 1]])
x = np.linspace(-3, 3, 200)
y = np.linspace(-3, 3, 200)
X, Y = np.meshgrid(x, y)
XY = np.empty(X.shape + (2,))
XY[:,:,0] = X; XY[:,:,1] = Y

In [ ]:
plt.contour(X, Y, p2d.pdf(XY))
plt.xlabel('x', fontsize=15)
plt.ylabel('y', fontsize=15)
plt.title('2D Gaussian', fontsize=15);

The problem now is that an isotropic distribution is not a good fit since the above density has two very different length scales. Having a tight fit between the sampling and target density becomes even more important in high dimensions.

For the sake of argument, consider the problem of sampling from a multi-variate Gaussian with mean zero and diagonal covariance matrix $\sigma_p^2 \mathbf{I}_D$ where $\mathbf{I}_D$ denotes the $D \times D$ identity matrix. As a sampling density, we choose another Gaussian with covariance $\sigma_q^2 \mathbf{I}_D$. The condition, $c q \geq p$ now requires that $\sigma_q^2 \geq \sigma_p^2$ and the optimal (smallest) value of $c$ is found to be $\left( \frac{\sigma_p}{\sigma_q} \right)^D$. Then, the acceptance probability is given by the ratio of volumes under $p(x)$ and $c q(x)$ which is just $\frac{1}{c}$ since both distributions are normalized. Thus, the acceptance probability vanishes exponentially in $D$.

**Exercise:** Illustrate this effect when $\sigma_q$ exceeds $\sigma_p$ by $1 \%$.

In [ ]:
# <api>
def gauss_hist2d (ax, data, bins):
    x = np.array(data)[:,0]
    y = np.array(data)[:,1]
    ax.hist2d(x, y, bins, normed=True)

In [ ]:
q = multivariate_normal(mean=[0,0], cov=np.eye(2))
sampling = RejectionSampling(p=p2d.pdf, q=q, c=p2d.pdf([0,0])/q.pdf([0,0]))

### How can we draw the expectation for 2D data?
smpl_intro.show_sampling(sampling, plotter=gauss_hist2d, exp=False)
print(sampling)

With a much higher rejection rate than in the 1-dimensional example above.

The next example shows that rejection sampling fails, i.e. samples from the wrong distribution, if the condition $c q(x) \geq p(x)$ is violated:

In [ ]:
sampling = RejectionSampling(p=p, q=norm(loc=0, scale=2), c=2)

smpl_intro.show_sampling(sampling, plotter=smpl_intro.gauss_hist, 
                    f_exp=lambda x: np.power(x,2), true_exp=1.)

print(sampling)

Comparing the empirical density of the samples with the desired distribution, it is clearly visible that values close to zero are under represented. Accordingly, the empirical expectation of $x^2$ is higher than it would be under the desired distribution.